# Load Pickle

In [1]:
import pickle
root_wd = '/Users/zakkyang/Documents/Data_Mining/Python/Projects/0_My_Project/Online_Shopper_Prediction/'
data= pickle.load(open(root_wd + 'Pickle/all.p', 'rb'))

In [2]:
predictors = []

# Feature Engineering

## Load Libraries

In [3]:
import pandas as pd
import numpy as np
import gc

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

## Merge nunique

In [4]:
data.head()

,account_view_pages,account_view_duration,info_view_pages,info_view_duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,0,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,0,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,0,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,0,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,1,0


In [5]:
def merge_nunique(df, columns_groupby, column, new_column_name, type='uint64'):
    add = pd.DataFrame(df.groupby(columns_groupby)[column].nunique()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    predictors.append(new_column_name)
    return df

In [6]:
data = merge_nunique(data, ['OperatingSystems'], 'Browser', 'nunique_Browser_gb_OS', 'uint32')
data = merge_nunique(data, ['OperatingSystems', 'Browser'], 'Region', 'nunique_region_gb_OB', 'uint32')
data = merge_nunique(data, ['VisitorType'], 'OperatingSystems', 'nunique_OS_gb_VisitorType', 'uint32')
data = merge_nunique(data, ['VisitorType', 'OperatingSystems'], 'Browser', 'nunique_Browser_gb_VO', 'uint32')
data = merge_nunique(data, ['VisitorType'], 'TrafficType', 'nunique_TrafficType_gb_VisitorType', 'uint32')
gc.collect()

168

In [7]:
data.columns

Index(['account_view_pages', 'account_view_duration', 'info_view_pages',
       'info_view_duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend', 'Revenue', 'nunique_Browser_gb_OS', 'nunique_region_gb_OB',
       'nunique_OS_gb_VisitorType', 'nunique_Browser_gb_VO',
       'nunique_TrafficType_gb_VisitorType'],
      dtype='object')

## Merge Cumulative Counts

In [8]:
def merge_cumcount(df, columns_groupby, column, new_column_name, type='uint64'):
    df[new_column_name] = df.groupby(columns_groupby)[column].cumcount().values.astype(type)
    predictors.append(new_column_name)
    return df

In [9]:
data = merge_cumcount(data, ['VisitorType'], 'TrafficType', 'cumcount_TrafficType_gb_VisitorType', 'uint32');
data = merge_cumcount(data, ['OperatingSystems', 'Browser'], 'Region', 'cumcount__Region_gb_OB', 'uint32');
data = merge_cumcount(data, ['VisitorType'], 'PageValues', 'cumcount_PageValues_gb_VisitorType', 'float32')

gc.collect()

14

## Merge Count

In [10]:
#similar to merge_unique
def merge_count(df, columns_groupby, new_column_name, type='uint64'):
    add = pd.DataFrame(df.groupby(columns_groupby).size()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    predictors.append(new_column_name)
    return df

In [11]:
data.columns

Index(['account_view_pages', 'account_view_duration', 'info_view_pages',
       'info_view_duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend', 'Revenue', 'nunique_Browser_gb_OS', 'nunique_region_gb_OB',
       'nunique_OS_gb_VisitorType', 'nunique_Browser_gb_VO',
       'nunique_TrafficType_gb_VisitorType',
       'cumcount_TrafficType_gb_VisitorType', 'cumcount__Region_gb_OB',
       'cumcount_PageValues_gb_VisitorType'],
      dtype='object')

In [12]:
data = merge_count(data, ['Month', 'SpecialDay'], 'count_gb_Month_Day', 'uint32');
data = merge_count(data, ['VisitorType', 'TrafficType', 'Region'], 'count_gb_VT_TT_region', 'uint32');

gc.collect()

56

## Merge Variance

In [13]:
def merge_var(df, columns_groupby, column, new_column_name, type='float64'):
    add = pd.DataFrame(df.groupby(columns_groupby)[column].var()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    predictors.append(new_column_name)
    return df

In [14]:
data = merge_var(data, ['VisitorType', 'OperatingSystems', 'Browser'], 'SpecialDay', 'var_day_gb_VT_OS_BW', 'float32')

## Merge Sum

In [15]:
def merge_sum(df, columns_groupby, column, new_column_name, type='float64'):
    add = pd.DataFrame(df.groupby(columns_groupby)[column].sum()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    # predictors.append(new_column_name)  # bug: twice
    return df

In [16]:
data = merge_sum(data, ['VisitorType'], 'PageValues', 'sum_PageValues_gb_VisitorType', 'float32')

## One-Hot Encoding

In [17]:
#one-hot coding to Month
def encode_onehot(df, column_name):
    df_onehot = pd.get_dummies(df[column_name], prefix=column_name)
    df_all = pd.concat([df.drop([column_name], axis=1), df_onehot], axis=1)
    predictors.append(column_name)
    return df_all

data=encode_onehot(data, 'Month')
data=encode_onehot(data, 'Browser')
data=encode_onehot(data, 'Region')
data=encode_onehot(data, 'VisitorType')

In [18]:
data.shape

(12330, 61)

In [19]:
predictors

['nunique_Browser_gb_OS',
 'nunique_region_gb_OB',
 'nunique_OS_gb_VisitorType',
 'nunique_Browser_gb_VO',
 'nunique_TrafficType_gb_VisitorType',
 'cumcount_TrafficType_gb_VisitorType',
 'cumcount__Region_gb_OB',
 'cumcount_PageValues_gb_VisitorType',
 'count_gb_Month_Day',
 'count_gb_VT_TT_region',
 'var_day_gb_VT_OS_BW',
 'Month',
 'Browser',
 'Region',
 'VisitorType']

## Drop Features

In [20]:
drop_features = ['TrafficType', 'OperatingSystems']

In [21]:
data.drop(columns=drop_features, inplace=True)

# Save Pickle

In [22]:
pickle.dump(data, open(root_wd + '/Pickle/feature_completed.p', 'wb'))